<pre>
<img align="center" width="300" src="https://portal.coren-sp.gov.br/wp-content/uploads/2014/11/logo%20uninove.png">
</pre>

<pre>
<img align="left" width="150" src="https://camo.githubusercontent.com/bdc6a3b8963aa99ff57dfd6e1e4b937bd2e752bcb1f1936f90368e5c3a38f670/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f4c6963656e73652d434325323042592d2d5341253230342e302d6c69676874677265792e737667">
</pre>

* __PPGI - UNINOVE__
* Computação Científica com Julia
* Prof. José E. Storópoli
* Aluno: André Santos

In [10]:
# Packages requires to load the dataset
using Pkg
Pkg.add("DataFrames")
using DataFrames

   Resolving package versions...
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\andre\.julia\environments\v1.7\Manifest.toml`


In [11]:
# Packages installed
Pkg.status()

      Status `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1
  [a93c6f00] DataFrames v1.3.3
  [41a02a25] Folds v0.2.8
  [38e38edf] GLM v1.7.0
  [7073ff75] IJulia v1.23.3
  [91a5bcdd] Plots v1.25.9
  [c3e4b0f8] Pluto v0.19.3


In [12]:
date = [1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,
    2012,2013,2014,2015,2016,2017,2018,2019];
    
pop = [2,1,4,2,1,4,2,2,1,6,7,7,8,8,8,8,11,11,13,21,21,36,24,41,46,46,59,53];
df = DataFrame( varx = date, vary = pop);

## Get the sum of the squares of the residuals (total, before and after break point)

In [13]:
# Define time series for Chow test:
#point break = 2010
start_df1, end_df1 = 1992, 2010                                      # range for series 1 (obs:"end_df1 = point break")
start_df2, end_df2 = end_df1+1, 2019                                 # range for series 2
df1 = filter(r -> r.varx >= start_df1 && r.varx <= end_df1, df)      # time series before point break
df2 = filter(r -> r.varx >= start_df2 && r.varx <= end_df2, df)      # time series after point break

,varx,vary
,Int64,Int64
1,2011,21
2,2012,21
3,2013,36
4,2014,24
5,2015,41
6,2016,46
7,2017,46
8,2018,59
9,2019,53


In [14]:
# Variables for Chow test:
x , y = copy(df[:,:varx]), copy(df[:,:vary])         # variables from original dataset
x1 , y1 = copy(df1[:,:varx]), copy(df1[:,:vary])     # variables from series before point break
x2 , y2 = copy(df2[:,:varx]), copy(df2[:,:vary])     # variables from series after point break

([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], [21, 21, 36, 24, 41, 46, 46, 59, 53])

In [15]:
# Alternative formulas ***************************************************************************************************
# Original dataset ----------------------------------------------------------------------------------------------- 
k = 2                                        # number of samples (series before and after point break)
n = length(x)                                # total scores from original dataset
sum_x, sum_y = sum(x), sum(y)                # sum of independent and dependent variables from original series
mean_x, mean_y = sum(x)/n, sum(y)/n          # average of independent and dependent variables from original series
SSxy = sum((x.-mean_x).*(y.-mean_y))         # covariance of X and Y
SSxx = sum((x.-mean_x).^2)                   # variance in X
b = SSxy/SSxx                                # regression coefficient
a = mean_y - b*mean_x                        # intercept
yi = a .+ b.*x                               # estimated (or predicted) y value for observation i
RSSp = sum((y .- yi).^2)                     # Resiual sum of square (errors)
# Series 1 ------------------------------------------------------------------------------------------------------ 
n1 = length(x1)                              # total scores from series 1
gl1 = n1 - k                                 # numerator degrees freedom
sum_x1, sum_y1 = sum(x1), sum(y1)            # sum of independent and dependent variables from series 1
mean_x1, mean_y1 = sum(x1)/n1, sum(y1)/n1    # average of independent and dependent variables from series 1
SSxy1 = sum((x1.-mean_x1).*(y1.-mean_y1))    # covariance of X1 and Y1
SSxx1 = sum((x1.-mean_x1).^2)                # variance in X1
b1 = SSxy1/SSxx1                             # regression coefficient (series 1)
a1 = mean_y1 - b1*mean_x1                    # intercept (series 1)
yi1 = a1 .+ b1.*x1                           # estimated y value for observation i (series 1)
RSS1 = sum((y1 .- yi1).^2)                   # Residual sum of square before break
# Series 2 ------------------------------------------------------------------------------------------------------ 
n2 = length(x2)                              # total scores from series 2
gl2 = n2 - k                                 # denominator degrees freedom
sum_x2, sum_y2 = sum(x2), sum(y2)            # sum of independent and dependent variables from series 2
mean_x2, mean_y2 = sum(x2)/n2, sum(y2)/n2    # average of independent and dependent variables from series 2
SSxy2 = sum((x2.-mean_x2).*(y2.-mean_y2))    # covariance of X2 and Y2
SSxx2 = sum((x2.-mean_x2).^2)                # variance in X2
b2 = SSxy2/SSxx2                             # regression coefficient (series 2)
a2 = mean_y2 - b2*mean_x2                    # intercept (series 2)
yi2 = a2 .+ b2.*x2                           # estimated y value for observation i (series 2)
RSS2 = sum((y2 .- yi2).^2)                   # Residual sum of square after break

233.95555555557337

## [Data parallelism](https://juliafolds.github.io/data-parallelism/tutorials/quick-introduction/)

In [3]:
#Pkg.add("Folds")

    Updating registry at `C:\Users\andre\.julia\registries\General.toml`
   Resolving package versions...
   Installed Accessors ────────── v0.1.11
   Installed ZygoteRules ──────── v0.2.2
   Installed DefineSingletons ─── v0.1.2
   Installed ArgCheck ─────────── v2.3.0
   Installed MicroCollections ─── v0.1.2
   Installed Referenceables ───── v0.1.2
   Installed InitialValues ────── v0.3.1
   Installed BangBang ─────────── v0.3.36
   Installed ConstructionBase ─── v1.3.0
   Installed Setfield ─────────── v0.8.2
   Installed CompositionsBase ─── v0.1.1
   Installed Baselet ──────────── v0.1.1
   Installed ExternalDocstrings ─ v0.1.1
   Installed ThreadedScans ────── v0.1.0
   Installed SplittablesBase ──── v0.1.14
   Installed Folds ────────────── v0.2.8
   Installed Transducers ──────── v0.4.73
    Updating `C:\Users\andre\.julia\environments\v1.7\Project.toml`
  [41a02a25] + Folds v0.2.8
    Updating `C:\Users\andre\.julia\environments\v1.7\Manifest.toml`
  [7d9f7c33] + Accessors v0.

In [5]:
using Folds

In [17]:
# Return regression coefficient
function mat_coeff(y, x)
    k = 2
    n = length(x)
    sum_x, sum_y = sum(x), sum(y)
    mean_x, mean_y = sum(x)/n, sum(y)/n
    SSxy = sum((x.-mean_x).*(y.-mean_y))
    SSxx = sum((x.-mean_x).^2)
    b = SSxy/SSxx
    return(b)
end

mat_coeff (generic function with 1 method)

In [21]:
# Return regression coefficient
function mat_coeff1(y, x)
    k = 2
    n = length(x)
    sum_x, sum_y = sum(x), sum(y)
    mean_x, mean_y = sum(x)/n, sum(y)/n
    SSxy = Folds.sum((x.-mean_x).*(y.-mean_y))  # parallelism
    SSxx = Folds.sum((x.-mean_x).^2)            # parallelism
    b = SSxy/SSxx
    return(b)
end

mat_coeff1 (generic function with 1 method)

In [27]:
# Results:
println(mat_coeff(y, x))
println(mat_coeff1(y, x))

1.889709906951286
1.889709906951286


## [BenchmarkTools.jl](https://juliaci.github.io/BenchmarkTools.jl/stable/)

In [28]:
# Packages requires to load the dataset
#using Pkg
#Pkg.add("BenchmarkTools")
using BenchmarkTools

In [29]:
@btime mat_coeff(y, x)

  225.636 ns (3 allocations: 592 bytes)


1.889709906951286

In [30]:
@btime mat_coeff1(y, x)

  367.647 ns (5 allocations: 624 bytes)


1.889709906951286

In [31]:
@benchmark mat_coeff(y,x)

BenchmarkTools.Trial: 10000 samples with 535 evaluations.
 Range (min … max):  214.393 ns …   4.547 μs  ┊ GC (min … max): 0.00% … 89.25%
 Time  (median):     239.252 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   312.144 ns ± 292.453 ns  ┊ GC (mean ± σ):  7.33% ±  7.53%

  ▇█▇▅▅▄▃▂▁▂▂▂▁  ▁     ▁▁▂▃▄▄▅▅▄▄▄▃▂▁                           ▂
  █████████████████████████████████████▇▇▆▆▅▅▆▇▆▇█▆▆▇▅▅▅▅▄▅▆▅▅▅ █
  214 ns        Histogram: log(frequency) by time        581 ns <

 Memory estimate: 592 bytes, allocs estimate: 3.

In [32]:
@benchmark mat_coeff1(y,x)

BenchmarkTools.Trial: 10000 samples with 211 evaluations.
 Range (min … max):  368.246 ns …  14.390 μs  ┊ GC (min … max): 0.00% … 94.70%
 Time  (median):     419.431 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   558.870 ns ± 594.087 ns  ┊ GC (mean ± σ):  5.24% ±  4.98%

  █▇▅▄▄▄▄▃▁▁             ▁▁▂▃▄▅▅▅▅▄▄▃▂▂▁                        ▂
  ████████████▇▇█▇▆▅▅▆▇▇▇████████████████▇▆▆▅▆▅▃▁▁▄▃▃▄▄▄▄▁▃▄▆██ █
  368 ns        Histogram: log(frequency) by time       1.07 μs <

 Memory estimate: 624 bytes, allocs estimate: 5.

### Conclusão

A função que utilizou paralelismo usou 32 bytes (624-592) de memória a e foi mais lenta em 142.011 nano segundos (367.647-225.636).

### Referências

* [Ciência de Dados e Computação Científica com Julia](https://storopoli.io/Computacao-Cientifica/2_BenchmarkTools/)
* [BenchmarkTools.jl](https://juliaci.github.io/BenchmarkTools.jl/stable/)
* [juliastats.org](https://juliastats.org/GLM.jl/stable/examples/)
* [Julia - GLM Documentation](https://juliastats.org/GLM.jl/v0.11/)
* [Ordens de magnitude para tempo](https://pt.wikipedia.org/wiki/Ordens_de_magnitude_para_tempo)
* [Github Chow_Test Julia](https://github.com/andrelmfsantos/Chow_Test/blob/main/Chow%20Test%20Julia%20-%20Atelier.ipynb)
* [First-time setup: Install Julia & Pluto](https://computationalthinking.mit.edu/Fall20/installation/)
* [Data-parallel Julia](https://juliafolds.github.io/data-parallelism/tutorials/quick-introduction/)
* [JuliaFolds](https://github.com/JuliaFolds)